# Start Simulation

In [1]:
import random
import numpy as np
import os
import time
from PIL import Image

import torch
torch.__version__
device = torch.device("cuda")

In [2]:
import getpass
user = getpass.getuser()
print(user)

yizhou


In [3]:
usd_path = f"omniverse://localhost/Users/{user}/uva_sac.usd"

In [4]:
from omni.isaac.kit import SimulationApp    
simulation_app = SimulationApp({"headless": True, "open_usd": usd_path,  "livesync_usd": usd_path}) 

Starting kit application with the fillowing args:  ['/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/exts/omni.isaac.kit/omni/isaac/kit/simulation_app.py', '/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/apps/omni.isaac.sim.python.kit', '--/app/tokens/exe-path=/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/kit', '--/persistent/app/viewport/displayOptions=3094', '--/rtx/materialDb/syncLoads=True', '--/rtx/hydra/materialSyncLoads=True--/omni.kit.plugin/syncUsdLoads=True', '--/app/renderer/resolution/width=1280', '--/app/renderer/resolution/height=720', '--/app/window/width=1440', '--/app/window/height=900', '--/renderer/multiGpu/enabled=True', '--ext-folder', '/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/exts', '--ext-folder', '/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/apps', '--/physics/cudaDevice=0', '--portable', '--no-window']
Passing the following args to the base kit application:  ['-f', '/home/yizhou/.local/share/jupyter/runtime/kernel-2fed25f9-a2d9-

OmniAssetFileFormat


Warp initialized:
   Version: 0.2.2
   CUDA device: NVIDIA GeForce RTX 3090
   Kernel cache: /home/yizhou/.cache/warp/0.2.2
[1.890s] [ext: omni.kit.menu.create-1.0.2] startup
[1.891s] [ext: omni.mdl-0.1.0] startup
[1.914s] [ext: omni.kit.window.file_importer-1.0.4] startup
[1.915s] [ext: omni.kit.window.file_exporter-1.0.4] startup
[1.917s] [ext: omni.kit.material.library-1.3.10] startup
[1.922s] [ext: omni.kit.window.drop_support-1.0.0] startup
[1.923s] [ext: omni.kit.window.file-1.3.16] startup
[1.925s] [ext: omni.kit.context_menu-1.3.9] startup
[1.929s] [ext: omni.kit.window.property-1.6.3] startup
[1.930s] [ext: omni.kit.window.content_browser-2.4.28] startup
[1.941s] [ext: omni.kit.widget.stage-2.6.15] startup
[1.945s] [ext: omni.isaac.version-1.0.0] startup
[1.946s] [ext: omni.kit.property.usd-3.14.9] startup
[1.981s] [ext: omni.kit.viewport.legacy_gizmos-1.0.0] startup
[1.984s] [ext: omni.hydra.rtx-0.1.0] startup
[1.995s] [ext: omni.renderer-rtx-0.0.0] startup
[1.995s] [ext: omn

In [5]:
# set log level
import logging
import carb

logging.getLogger("omni.hydra").setLevel(logging.ERROR)
logging.getLogger("omni.isaac.urdf").setLevel(logging.ERROR)
logging.getLogger("omni.physx.plugin").setLevel(logging.ERROR)

logging.getLogger("omni.isaac.synthetic_utils").setLevel(logging.ERROR)
logging.getLogger("omni.isaac.synthetic_utils.syntheticdata").setLevel(logging.ERROR)
logging.getLogger("omni.hydra.scene_delegate.plugin").setLevel(logging.ERROR)


l = carb.logging.LEVEL_ERROR
carb.settings.get_settings().set("/log/level", l)
carb.settings.get_settings().set("/log/fileLogLevel", l)
carb.settings.get_settings().set("/log/outputStreamLevel", l)

# # This logged error is printed as it should
# carb.log_error("ERROR")
# # This warning is printed but should not
# carb.log_warn("WARNING")

# Config

In [6]:
from task.config import DATA_PATH, FEATURE_PATH
task_type = "Table"
side_choice = "Border"
base_asset_id = 0
load_nucleus = True

In [7]:
pause

NameError: name 'pause' is not defined

# Init Env

In [ ]:
from uva_env import UvaEnv
env = UvaEnv()

In [ ]:
from task.utils import add_scene_default
add_scene_default()

In [ ]:
print(list(env.stage.TraverseAll()))

In [ ]:
env.clean()
env.world.step(render=True)

# Scene

In [ ]:
from task.scene import ArrangeScene
scene = ArrangeScene(task_type, side_choice, base_asset_id = 0, traj_id = 0, load_nucleus = load_nucleus)
env.scene = scene

In [ ]:
# add base
scene.add_base_asset()

In [ ]:
# add room
# scene.add_room()

In [ ]:
env.world.step(render=True)

# Reward

In [ ]:
from uv.reward import Rewarder

In [ ]:
rewarder = Rewarder(env.world)
env.rewarder = rewarder

# Render

In [ ]:
from render.helper import RenderHelper
render = RenderHelper(task_type, side_choice)

render.add_task_cameras()
render.set_cameras()

In [ ]:
pause

#  Learning

In [ ]:
# from learning.network.resnet import ResNetFeatureExtractor

# feature extraction
from learning.utils import extract_image_clip_feature_and_save, obtain_action_from_trainer
from transformers import CLIPProcessor, CLIPModel

feature_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
feature_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [ ]:
# replay buffer
import json
from learning.replay_buffer import ReplayBuffer
from learning.config import *

buffer = ReplayBuffer(max_size=2000)

In [ ]:
# trainer
from learning.network.sac import *

policy = Policy()

qf1 = QFunction()
qf2 = QFunction()
target_qf1 = QFunction()
target_qf2 = QFunction()

from learning.sac_trainer import SACTrainer

trainer = SACTrainer(policy, qf1, qf2, target_qf1, target_qf2, 
     use_automatic_entropy_tuning = False, 
     policy_lr=1e-3, 
     qf_lr=1e-3,
     target_update_period = 5)

# Trajectory

In [ ]:
use_network = True
debug = True

In [ ]:
total_traj = 0
total_step = 0

---------------------

In [ ]:
# traj config
for traj_id in range(200):
    total_traj += 1
    
    base_asset_id = 0
    env.scene.base_asset_id = base_asset_id
    env.scene.traj_id = traj_id
    image_folder = os.path.join(DATA_PATH, task_type, side_choice, str(traj_id))

    # base
    # scene.add_base_asset()
    env.world.step(render = True)

    # get images
    env.world.render()
    images = render.get_images()
    render.save_rgb(images[0]['rgb'], image_folder, "0")
    
    ## extract feature
    
#     extract_image_feature_and_save(images[0]['rgb'][:,:,:3], 
#         feature_extractor, os.path.join(image_folder, str(0) + ".pt"))
    extract_image_clip_feature_and_save(f"{image_folder}/{0}.png", feature_model, feature_processor, 
        f"{image_folder}/{0}.pt",)
    

    # trajectory
    for step in range(5):
        total_step += 1
        
        # sample an object
        env.add_scene_obj(mode = "random")
        
        # TODO: get action from sampling
        if not use_network or total_traj < 10 or np.random.rand() < 0.2:
            x, y = np.tanh(np.random.randn()), np.tanh(np.random.randn())
        else:
            image_feature_file = f"{image_folder}/{step}.pt"
            
            object_info = env.scene.objects[-1]
            object_type = object_info["type"]
            obj_name = object_info["name"][:-4]
            object_feature_file = os.path.join(FEATURE_PATH, object_type, obj_name + ".pt")
            x, y = obtain_action_from_trainer(image_feature_file, object_feature_file, trainer)
        
        # load the object into the scene
        env.put_last_object((x, y)) 
        env.world.step(render=True)
        
        # register the object to the world for physics update
        env.register_last_object()
        env.world.step(render=True)

        # get images
        env.world.render()
        images = render.get_images()
        render.save_rgb(images[0]['rgb'], image_folder, str(step + 1))

        ## calculate reward
        env.calculate_last_reward(simulation_step = 30)
        
        ## extract feature
#         extract_image_feature_and_save(images[0]['rgb'][:,:,:3], 
#             feature_extractor, os.path.join(image_folder, str(step + 1) + ".pt"))
        extract_image_clip_feature_and_save(f"{image_folder}/{step + 1}.png", feature_model, feature_processor, 
            f"{image_folder}/{step + 1}.pt",)
    

        ## reset
        env.world.reset()
        env.world.step(render=True)
        
        ## trainer nework
        if use_network and total_step % UPDATE_TRAINER_STEPS == 0 and total_traj > 5:
            batch = buffer.sample_batch(batch_size = BATCH_SIZE)
            trainer.update(batch)
            
            if debug:
                rewards = batch['rewards'].to(device)
                terminals = batch['terminals'].to(device)
                obs = batch['observations'].to(device)
                actions = batch['actions'].to(device)
                next_obs = batch['next_observations'].to(device)
                obj_features = batch['object_features'].to(device)
                
                dist = trainer.policy(obs, obj_features)
                pred = trainer.qf1(obs, obj_features, actions)
                print(f"debug {total_traj}/{total_step}", #"\n dist: ", dist.mean.flatten().tolist(), dist.stddev.flatten().tolist(),
                      "\n pred:", pred.flatten().tolist(),
                      "\n rewards: ", rewards.flatten().tolist())


    # Record
    record = env.scene.get_scene_data()
    env.scene.save_scene_data()
    # print("record: ", record)
    
    # Add record to buffer
    buffer.add_scene_sample(record)

    # Reset (env clean)
    env.clean(clean_all = False)
    env.step(render = True)

# Debug

In [ ]:
# get images
env.world.render()
env.world.render()
images = render.get_images()

Image.fromarray(images[0]['rgb'], "RGBA").resize((300, 300))

##  Debug Training

In [8]:
import json
from learning.replay_buffer import ReplayBuffer

buffer = ReplayBuffer(max_size=1000)

for i in range(100):
    replay = json.load(open(f"{DATA_PATH}/{task_type}/{side_choice}/{i}/scene.json"))
    buffer.add_scene_sample(replay)

In [9]:
len(buffer.dataset)

500

In [10]:
# trainer
from learning.network.sac import *

policy = Policy()

qf1 = QFunction()
qf2 = QFunction()

target_qf1 = QFunction()
target_qf1.eval()

target_qf2 = QFunction()
target_qf2.eval()

from learning.sac_trainer import SACTrainer

trainer = SACTrainer(policy, qf1, qf2, target_qf1, target_qf2, 
     use_automatic_entropy_tuning = True, 
     policy_lr=1e-3, 
     qf_lr=1e-3,
     target_update_period = 5)

In [11]:
self = trainer

In [13]:
for _ in range(500):
    batch = buffer.sample_batch(batch_size=6)

    rewards = batch['rewards'].to(device)
    terminals = batch['terminals'].to(device)
    obs = batch['observations'].to(device)
    actions = batch['actions'].to(device)
    next_obs = batch['next_observations'].to(device)

    obj_features = batch['object_features'].to(device)

    # print("rewards", rewards)

    self.update(batch)
    
    if _ % 10 == 0:
        dist = self.policy(obs, obj_features)
        pred = self.qf1(obs, obj_features, actions)
        print(_, "\n dist   : ", [round(x, 3) for x in dist.mean.flatten().tolist()], 
                  [round(x, 3) for x in dist.stddev.flatten().tolist()],
              "\n pred   :", [round(x, 3) for x in pred.flatten().tolist()],
              "\n rewards: ", [round(x, 3) for x in rewards.flatten().tolist()])
    
#     # debug
#     pred = self.qf1(obs, obj_features, actions)
#     loss = self.qf_criterion(pred, rewards)
    
#     self.qf1_optimizer.zero_grad()
#     loss.backward()
#     self.qf1_optimizer.step()
    
#     if _ % 20 == 0:
#         print(_, "\n debug loss: ", loss.item(), "\n pred:", 
#               pred.flatten().tolist(), "\n rewards: ", rewards.flatten().tolist())

0 
 dist   :  [0.108, 0.162, 0.131, 0.232, 0.12, 0.253, 0.145, 0.269, 0.148, 0.285, 0.14, 0.282] [0.715, 0.722, 0.637, 0.664, 0.617, 0.627, 0.586, 0.598, 0.568, 0.587, 0.575, 0.598] 
 pred   : [-0.491, -0.205, -0.287, 0.446, -0.151, -0.207] 
 rewards:  [-0.839, -0.839, -0.832, 1.0, -0.839, -0.839]
10 
 dist   :  [0.213, 0.162, 0.187, 0.148, 0.24, 0.124, 0.116, 0.191, 0.19, 0.154, 0.256, 0.189] [0.541, 0.501, 0.585, 0.558, 0.583, 0.553, 0.564, 0.537, 0.576, 0.551, 0.508, 0.495] 
 pred   : [1.274, 1.34, -0.209, 0.233, -0.079, -0.356] 
 rewards:  [1.0, 0.899, -0.839, -0.839, -0.839, -0.839]
20 
 dist   :  [0.136, 0.121, 0.12, 0.117, 0.092, 0.113, 0.114, 0.135, 0.102, 0.139, 0.093, 0.078] [0.518, 0.471, 0.55, 0.509, 0.543, 0.47, 0.519, 0.471, 0.503, 0.441, 0.625, 0.579] 
 pred   : [-0.297, -0.125, 0.389, 1.293, -0.196, -0.089] 
 rewards:  [-0.839, -0.839, -0.839, 0.953, -0.839, -0.839]
30 
 dist   :  [0.047, 0.366, 0.103, 0.232, 0.103, 0.316, 0.105, 0.328, 0.119, 0.239, -0.013, 0.326] [0.4

280 
 dist   :  [0.221, 0.278, 0.214, 0.291, 0.22, 0.309, 0.241, 0.304, 0.247, 0.389, 0.177, 0.323] [0.556, 0.55, 0.547, 0.523, 0.523, 0.497, 0.531, 0.516, 0.418, 0.414, 0.49, 0.478] 
 pred   : [1.295, 0.303, 0.953, 0.239, 0.269, 1.986] 
 rewards:  [0.984, -0.839, -0.839, -0.796, 0.485, 1.0]
290 
 dist   :  [0.149, 0.316, 0.159, 0.348, 0.171, 0.322, 0.149, 0.29, 0.106, 0.21, 0.152, 0.303] [0.485, 0.444, 0.45, 0.408, 0.486, 0.445, 0.528, 0.484, 0.639, 0.601, 0.501, 0.464] 
 pred   : [-0.112, 1.381, 0.038, -0.188, 1.144, 0.209] 
 rewards:  [-0.839, 0.999, -0.839, -0.839, 0.999, -0.839]
300 
 dist   :  [0.014, 0.364, -0.023, 0.433, -0.001, 0.302, -0.015, 0.356, -0.046, 0.441, -0.013, 0.347] [0.42, 0.379, 0.338, 0.304, 0.5, 0.448, 0.439, 0.421, 0.331, 0.296, 0.449, 0.392] 
 pred   : [0.791, -0.383, -0.299, 1.51, 1.678, -0.324] 
 rewards:  [0.358, -0.839, -0.839, 0.997, 1.0, -0.839]
310 
 dist   :  [-0.058, 0.259, -0.054, 0.27, -0.034, 0.356, -0.05, 0.269, -0.078, 0.404, -0.039, 0.303] [0.4

In [ ]:
dist = self.policy(obs.to(device), obj_features.to(device))

In [ ]:
dist.sample()